# API/Control, Backend, OOP, Python
> Between Model and View is Control.  In this Web application it will be handled by APIs.  Data will be input and displayed through View and stored in Model.  Control will broker information between projects.

- title: API/Control, Backend, OOP, Python
- toc: true
- categories: []
- image: /images/python.png
- type: pbl
- week: 19

## Database and SQLAlchemy

This lesson is to help you build your database for your group passion project and set up a data table to store the log-in information of your users. 

Databases, Iterative/OOP
- Iterative: Refers to a sequence of instructions or code being repeated until a specific end result is achieved
- OOP: A computer programming model that organizes software design around data, or objects, rather than functions and logic
- SQL: Structured Query Language, abbreviated as SQL, is a language used in programming, managing, and structuring data

**In order to build your log-in system using the following code, you will need to first download an extension: "sqlite viewer"**

In [ ]:
"""
These imports define the key objects
"""

from flask import Flask
from flask_sqlalchemy import SQLAlchemy

"""
These object and definitions are used throughout the Jupyter Notebook.
"""

# Setup of key Flask object (app)
app = Flask(__name__)
# Setup SQLAlchemy object and properties for the database (db)
database = 'sqlite:///sqlite.db'  # path and filename of database
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SQLALCHEMY_DATABASE_URI'] = database
app.config['SECRET_KEY'] = 'SECRET_KEY'
db = SQLAlchemy()


# This belongs in place where it runs once per project
db.init_app(app)

### Model Definition

- Define columns, initialization, and CRUD methods for users table in sqlite.db

This is the model code, or the actual code that will initialize your data table and assign properties to your columns. This table will store the data users input when building thier log-ins to your site. 

In [ ]:
""" database dependencies to support sqlite examples """
import datetime
from datetime import datetime
import json

from sqlalchemy.exc import IntegrityError
from werkzeug.security import generate_password_hash, check_password_hash


''' Tutorial: https://www.sqlalchemy.org/library.html#tutorials, try to get into a Python shell and follow along '''

# Define the User class to manage actions in the 'users' table
# -- Object Relational Mapping (ORM) is the key concept of SQLAlchemy
# -- a.) db.Model is like an inner layer of the onion in ORM
# -- b.) User represents data we want to store, something that is built on db.Model
# -- c.) SQLAlchemy ORM is layer on top of SQLAlchemy Core, then SQLAlchemy engine, SQL
class User(db.Model):
    __tablename__ = 'users'  # table name is plural, class name is singular

    # Define the User schema with "vars" from object
    id = db.Column(db.Integer, primary_key=True)
    _name = db.Column(db.String(255), unique=False, nullable=False)
    _uid = db.Column(db.String(255), unique=True, nullable=False)
    _password = db.Column(db.String(255), unique=False, nullable=False)
    _dob = db.Column(db.Date)

    # constructor of a User object, initializes the instance variables within object (self)
    def __init__(self, name, uid, password="123qwerty", dob=datetime.today()):
        self._name = name    # variables with self prefix become part of the object, 
        self._uid = uid
        self.set_password(password)
        if isinstance(dob, str):  # not a date type     
            dob = date=datetime.today()
        self._dob = dob

    # a name getter method, extracts name from object
    @property
    def name(self):
        return self._name
    
    # a setter function, allows name to be updated after initial object creation
    @name.setter
    def name(self, name):
        self._name = name
    
    # a getter method, extracts uid from object
    @property
    def uid(self):
        return self._uid
    
    # a setter function, allows uid to be updated after initial object creation
    @uid.setter
    def uid(self, uid):
        self._uid = uid
        
    # check if uid parameter matches user id in object, return boolean
    def is_uid(self, uid):
        return self._uid == uid
    
    @property
    def password(self):
        return self._password[0:10] + "..." # because of security only show 1st characters

    # update password, this is conventional method used for setter
    def set_password(self, password):
        """Create a hashed password."""
        self._password = generate_password_hash(password, method='sha256')

    # check password parameter against stored/encrypted password
    def is_password(self, password):
        """Check against hashed password."""
        result = check_password_hash(self._password, password)
        return result
    
    # dob property is returned as string, a string represents date outside object
    @property
    def dob(self):
        dob_string = self._dob.strftime('%m-%d-%Y')
        return dob_string
    
    # dob setter, verifies date type before it is set or default to today
    @dob.setter
    def dob(self, dob):
        if isinstance(dob, str):  # not a date type     
            dob = date=datetime.today()
        self._dob = dob
    
    # age is calculated field, age is returned according to date of birth
    @property
    def age(self):
        today = datetime.today()
        return today.year - self._dob.year - ((today.month, today.day) < (self._dob.month, self._dob.day))
    
    # output content using str(object) is in human readable form
    # output content using json dumps, this is ready for API response
    def __str__(self):
        return json.dumps(self.read())

    # CRUD create/add a new record to the table
    # returns self or None on error
    def create(self):
        try:
            # creates a person object from User(db.Model) class, passes initializers
            db.session.add(self)  # add prepares to persist person object to Users table
            db.session.commit()  # SqlAlchemy "unit of work pattern" requires a manual commit
            return self
        except IntegrityError:
            db.session.remove()
            return None

    # CRUD read converts self to dictionary
    # returns dictionary
    def read(self):
        return {
            "id": self.id,
            "name": self.name,
            "uid": self.uid,
            "dob": self.dob,
            "age": self.age,
        }

    # CRUD update: updates user name, password, phone
    # returns self
    def update(self, name="", uid="", password=""):
        """only updates values with length"""
        if len(name) > 0:
            self.name = name
        if len(uid) > 0:
            self.uid = uid
        if len(password) > 0:
            self.set_password(password)
        db.session.add(self) # performs update when id exists
        db.session.commit()
        return self

    # CRUD delete: remove self
    # None
    def delete(self):
        db.session.delete(self)
        db.session.commit()
        return None

### Initial Data 

- Uses SQLALchemy db.create_all() to initialize rows into sqlite.db

This is for when you start running your flask server. It creates initial data in your database for testing. 

In [ ]:
"""Database Creation and Testing """


# Builds working data for testing
def initUsers():
    with app.app_context():
        """Create database and tables"""
        db.create_all()
        """Tester data for table"""
        u1 = User(name='Thomas Edison', uid='toby', password='123toby', dob=datetime(1847, 2, 11))
        u2 = User(name='Nikola Tesla', uid='niko', password='123niko')
        u3 = User(name='Alexander Graham Bell', uid='lex', password='123lex')
        u4 = User(name='Eli Whitney', uid='whit', password='123whit')
        u5 = User(name='Indiana Jones', uid='indi', dob=datetime(1920, 10, 21))
        u6 = User(name='Marion Ravenwood', uid='raven', dob=datetime(1921, 10, 21))


        users = [u1, u2, u3, u4, u5, u6]

        """Builds sample user/note(s) data"""
        for user in users:
            try:
                '''add user to table'''
                object = user.create()
                print(f"Created new uid {object.uid}")
            except:  # error raised if object nit created
                '''fails with bad or duplicate data'''
                print(f"Records exist uid {user.uid}, or error.")
                
initUsers()

### Check for given Credentials in users table in sqlite.db

- Use of ORM Query object and custom methods to identify user to credentials uid and password

This code allows you to search through your database to find individual users. This can be used to check if the username and password matches what the user inputted in the database. 

The first block of code uses SQRAlchemy's ORM capabilities to filter and retrieve a user based on their UID. The UID serves as the primary key for the 'users' table in the database.

The second block of code is focused on user authentication. It first checks if a user matching the given UID exists, then verifies if the password entered matches the stored password for that user. 

"sqlite.db" is the name of your file

In [ ]:
# SQLAlchemy extracts single user from database matching User ID
def find_by_uid(uid):
    with app.app_context():
        user = User.query.filter_by(_uid=uid).first()
    return user # returns user object

# Check credentials by finding user and verify password
def check_credentials(uid, password):
    # query email and return user record
    user = find_by_uid(uid)
    if user == None:
        return False
    if (user.is_password(password)):
        return True
    return False
        
#check_credentials("indi", "123qwerty")

### Create a new User in table in Sqlite.db

- Uses SQLALchemy and custom user.create() method to add row.

This is to allow a user to register an account on your site and make it so that their information goes straight to your data table when they do so. 

In [ ]:
# Inputs, Try/Except, and SQLAlchemy work together to build a valid database object
def create():
    # optimize user time to see if uid exists
    uid = input("Enter your user id:")
    user = find_by_uid(uid)
    try:
        print("Found\n", user.read())
        return
    except:
        pass # keep going
    
    # request value that ensure creating valid object
    name = input("Enter your name:")
    password = input("Enter your password")
    
    # Initialize User object before date
    user = User(name=name, 
                uid=uid, 
                password=password
                )
    
    # create user.dob, fail with today as dob
    dob = input("Enter your date of birth 'YYYY-MM-DD'")
    try:
        user.dob = datetime.strptime(dob, '%Y-%m-%d').date()
    except ValueError:
        user.dob = datetime.today()
        print(f"Invalid date {dob} require YYYY-mm-dd, date defaulted to {user.dob}")
           
    # write object to database
    with app.app_context():
        try:
            object = user.create()
            print("Created\n", object.read())
        except:  # error raised if object not created
            print("Unknown error uid {uid}")
        
create()

### Reading users table in sqlite.db

- Uses SQLALchemy query.all method to read data

This allows the computer to read all the data in your data table at once. You could use this function if you needed to print out all your data information, for example. It is also more efficient to upload all your data from the backend to frontend at once versus uploading each user's information individually, and this function allows you to do that.


In [ ]:
# SQLAlchemy extracts all users from database, turns each user into JSON
def read():
    with app.app_context():
        table = User.query.all()
    json_ready = [user.read() for user in table] # "List Comprehensions", for each user add user.read() to list
    return json_ready

read()

## API/Control Terms
> APIs work with methods to GET, POST, PUT, and UPDATE data.  Control helps with Requests, Response, and handling JSON.  Control is glue layer, thus the term Model-View-Control (or MVC).
- POST APIs interact with CREATE methods in Model.
- GET with READ
- PUT with UPDATE.
- DELETE with DELETE.

> During development it is best to work with Model and Control without involving View initially.   To support this type of development ...
- Become familiar with [Postman](https://www.postman.com/downloads/)
- FYI, as an alternative you can become familiar with working with APIs through ***curl***

### Resource
[API Docs](https://flask-restful.readthedocs.io/en/latest/api.html)

### Control/API code
> ***Control/API concepts*** are receiving and API request and returning a response.  

1. [Define API flask objects (api/user.py)](https://github.com/nighthawkcoders/flask_portfolio/blob/main/api/user.py#L7-L11).  Flask contains object that help in API definition.
2. [Register API objects (main.py)](https://github.com/nighthawkcoders/flask_portfolio/blob/main/main.py#L11-L23).  Every Flask object in the project needs to be registered with the "main" objects.
3. [Create/POST method](https://github.com/nighthawkcoders/flask_portfolio/blob/main/api/user.py#L14-L54).  Post method contains a lot of checking code, but ultimately it creates a database row in the Model and returns that row to the View.
4. [Read/GET method](https://github.com/nighthawkcoders/flask_portfolio/blob/main/api/user.py#L14-L54).  This shows off Object Relational Manager performing a User (class operation) to extract all records from the table and putting them into User objects.
5. [Define API endpoints](https://github.com/nighthawkcoders/flask_portfolio/blob/main/api/user.py#L62-L64).  Endpoints are somewhat patterns to be matched.  Note, url_prefix at top of file supplies "/api/users" prefix for each pattern.  Each pattern when matched invokes the correspond class.  Methods are defined in class to correspond to expectations (POST, GET, UPDATE, DELETE).


In [ ]:
from flask import Blueprint, request, jsonify
from flask_restful import Api, Resource # used for REST API building
from datetime import datetime

from model.users import User

# blueprint, which is registered to app in main.py
user_api = Blueprint('user_api', __name__,
                   url_prefix='/api/users')

# API docs https://flask-restful.readthedocs.io/en/latest/api.html#id1
api = Api(user_api)
class UserAPI:        
    class _Create(Resource):
        def post(self):
            ''' Read data for json body '''
            body = request.get_json()
            
            ''' Avoid garbage in, error checking '''
            # validate name
            name = body.get('name')
            if name is None or len(name) < 2:
                return {'message': f'Name is missing, or is less than 2 characters'}, 210
            # validate uid
            uid = body.get('uid')
            if uid is None or len(uid) < 2:
                return {'message': f'User ID is missing, or is less than 2 characters'}, 210
            # look for password and dob
            password = body.get('password')
            dob = body.get('dob')

            ''' #1: Key code block, setup USER OBJECT '''
            uo = User(name=name, 
                      uid=uid)
            
            ''' Additional garbage error checking '''
            # set password if provided
            if password is not None:
                uo.set_password(password)
            # convert to date type
            if dob is not None:
                try:
                    uo.dob = datetime.strptime(dob, '%m-%d-%Y').date()
                except:
                    return {'message': f'Date of birth format error {dob}, must be mm-dd-yyyy'}, 210
            
            ''' #2: Key Code block to add user to database '''
            # create user in database
            user = uo.create()
            # success returns json of user
            if user:
                return jsonify(user.read())
            # failure returns error
            return {'message': f'Processed {name}, either a format error or User ID {uid} is duplicate'}, 210

    class _Read(Resource):
        def get(self):
            users = User.query.all()    # read/extract all users from database
            json_ready = [user.read() for user in users]  # prepare output in json
            return jsonify(json_ready)  # jsonify creates Flask response object, more specific to APIs than json.dumps

    # building RESTapi endpoint
    api.add_resource(_Create, '/create')
    api.add_resource(_Read, '/')

### Testing APIs
> ***Backend Testing*** of APIs is best done through Browser for simple GET APIs, but other API methods (POST, UPDATE, DELETE) will require a tool like PostMan.

1. [Download Postman](https://www.postman.com/downloads/).  This tool test APIs effectively on localhost and is great aid for debugging.

2. [Main.py runtime configuration](https://github.com/nighthawkcoders/flask_portfolio/blob/main/main.py#L44-L47).  This configuration is setup to produce same port and localhost as deployment.

***`Run` locally as you develop*** Select main.py file in VSCode and press Play button, or press down arrow next to Play button to activate Debug testing.  The below dialog will appear in Terminal, though IP address will match you machines.

```bash
(base) machine:flask_portfolio user$  cd /Users/user/vscode/flask_portfolio ; /usr/bin/env /Users/user/opt/anaconda3/bin/python /Users/user/.vscode/extensions/ms-python.python-2022.20.2/pythonFiles/lib/python/debugpy/adapter/../../debugpy/launcher 61127 -- /Users/user/vscode/flask_portfolio/main.py 
 * Serving Flask app "__init__" (lazy loading)
 * Environment: production
   WARNING: This is a development server. Do not use it in a production deployment.
   Use a production WSGI server instead.
 * Debug mode: on
 * Running on all addresses.
   WARNING: This is a development server. Do not use it in a production deployment.
 * Running on http://192.168.1.75:8086/ (Press CTRL+C to quit)
 * Restarting with watchdog (fsevents)
 * Debugger is active!
 * Debugger PIN: 403-552-045
```

***`Test` API GET locally with Postman***.  Observe that tests may be saved.  
![](images/postmain_read_get_users.png)

***`Test` API POST locally with Postman***.  In this case, Postman can be used to add new records to the table.  Observe options to pass data using Body raw-json.
![](images/postman_create_post_users.png)

# Frontend code

In [ ]:
const apiUrl = 'http://localhost:8911/api/Login/'; // Replace with the actual API URL
        // Make an HTTP GET request to the API
fetch(apiUrl)
    .then(response => response.json()) // Parse the JSON response
    .then(data => {
        // Organize the data into a dictionary
        const organizedData = {
            Login_api: {
                url_prefix: '/api/Login',
                LoginAPI: {
                    get: {
                        description: 'Retrieve all Logins from the database',
                        url: '/api/Login',
                        method: 'GET',
                        data: data, // Include the retrieved data here
                        mode: "cors"
                    },
                },
            },
            LoginListAPI: {
                LoginListAPI: {
                    get: {
                        description: 'Retrieve all Logins from the database',
                        url: '/api/Login',
                        method: 'GET',
                        data: data, // Include the retrieved data here
                        mode: "cors"
                    },
                },
            },
        };
        // Now, you have the organized data in the "organizedData" dictionary
        Data = organizedData.Login_api.LoginAPI.get.data; //data[id].Login_name, image whatever u may need,

# Errors

## Not having SQLite

Windows :
    sudo apt -get install sqlite 3

Mac
    cope harder

## Python intepreter

Windows and mac:

    in vscode, ctrl/command shift p

    python; select interpreter
    
    make sure its which python

## if you have jwt module vs python (py JWT)

    uninstall and reinstall py jwt
